In [28]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
train_tape = "Z:/rppg/scamps_datatape_32x36x36_train.h5"
valid_tape = "Z:/rppg/scamps_datatape_32x36x36_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_deep_phys(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            #yield i, (j-tf.reduce_mean(j, axis=0))/(tf.math.reduce_std(j, axis=0)+1e-6)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_deep_phys(train).cache(f'{tmp}/deepphys'), to_tf_deep_phys(valid).cache(f'{tmp}/deepphys')
train, valid = to_tf_deep_phys(train), to_tf_deep_phys(valid)

In [30]:
deepphys = DeepPhys_end_to_end()
deepphys.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), loss='mse')
deepphys.build(input_shape=(None, 36, 36, 3))
print(f'Flops per frame:{get_flops(deepphys, input_sig=[tf.TensorSpec([32, 36, 36, 3])])/32:.0f}')
deepphys.summary()

Flops per frame:53434160
Model: "deep_phys_end_to_end_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_5 (Functional)         (None, 1)                 532899    
Total params: 532,899
Trainable params: 532,899
Non-trainable params: 0
_________________________________________________________________


In [31]:
deepphys.fit(train.shuffle(2000), validation_data=valid, epochs=3, callbacks=[keras.callbacks.ModelCheckpoint('../weights/DeepPhys_SCAMPS.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
deepphys.load_weights('../weights/DeepPhys_SCAMPS.h5')

Epoch 1/3
37909/37909 [==============================] - 1079s 28ms/step - loss: 0.1839 - val_loss: 0.1468

Epoch 00001: val_loss improved from inf to 0.14677, saving model to ../weights\DeepPhys_SCAMPS.h5
Epoch 2/3
37909/37909 [==============================] - 504s 13ms/step - loss: 0.1743 - val_loss: 0.1416

Epoch 00002: val_loss improved from 0.14677 to 0.14158, saving model to ../weights\DeepPhys_SCAMPS.h5
Epoch 3/3
37909/37909 [==============================] - 509s 13ms/step - loss: 0.1715 - val_loss: 0.1392

Epoch 00003: val_loss improved from 0.14158 to 0.13921, saving model to ../weights\DeepPhys_SCAMPS.h5


In [14]:
deepphys.load_weights('../weights/DeepPhys_SCAMPS.h5')

In [32]:
def cumsum(result, overwrite=True):
    with h5py.File(result, 'a') as f:
        for i, j in f.items():
            if 'diff' not in j.keys():
                j.create_dataset('diff', data=j['predict'])
                j['predict'][...] = detrend(np.cumsum(j['diff']))
            elif overwrite:
                j['predict'][...] = detrend(np.cumsum(j['diff']))

In [33]:
eval_on_dataset(test_set_PURE, deepphys, 32, (36, 36), step=1, batch=0, save='../results/DeepPhys_SCAMPS_PURE.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_SCAMPS_PURE.h5')
get_metrics('../results/DeepPhys_SCAMPS_PURE.h5')

100%|██████████| 59/59 [00:21<00:00,  2.71it/s]


{'Sliding window': {'MAE': 20.586, 'RMSE': 27.847, 'R': 0.20184},
 'Whole video': {'MAE': 16.97, 'RMSE': 24.572, 'R': 0.3114}}

In [34]:
eval_on_dataset(test_set_UBFC_rPPG2, deepphys, 32, (36, 36), step=1, batch=0, save='../results/DeepPhys_SCAMPS_UBFC.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_SCAMPS_UBFC.h5')
get_metrics('../results/DeepPhys_SCAMPS_UBFC.h5')

100%|██████████| 42/42 [00:14<00:00,  2.91it/s]


{'Sliding window': {'MAE': 15.621, 'RMSE': 22.903, 'R': 0.36048},
 'Whole video': {'MAE': 14.918, 'RMSE': 22.829, 'R': 0.3327}}